TODO: I will add a requirements file for this stuff

In [ ]:
! pip install smartnoise-synth

In [ ]:
! pip install git+https://github.com/ryan112358/private-pgm.git


In [ ]:
! pip install diffprivlib

In [ ]:
! pip install DataSynthesizer

## Use PrivateDataGenerator to generate datasets for each publication

In [3]:
from private_data_generator import PrivateDataGenerator
from lee2021ability.lee2021ability import Lee2021Ability

def dataframe_modifier_lee2021ability(df):
    df = df.replace({"Male": 0,
                     "Female": 1})
    df = df.replace({"White": 0,
                     "Black": 1,
                     "Hispanic": 2,
                     "Asian": 3,
                     "Other": 4,
                     "Other": 4})
    df = df

publication = Lee2021Ability(filename='lee2021ability_dataframe.pickle')
private_data_generator = PrivateDataGenerator(publication, dataframe_modifier)
private_data_generator.generate()

math          [99.1403, 72.4904, 75.4243, 60.028999999999996...
teacher       [2.2, 2.6, 1.6, 3.0, 2.4, 2.0, 1.4, 1.8, 2.8, ...
ability       [3.5, 3.25, 3.0, 4.0, 3.75, 2.25, 2.75, 2.5, 2...
parents       [0.6, 0.0, 0.8, 0.9, 0.5, 0.7, 0.2, 0.1, 0.4, ...
sex                                              [Male, Female]
race                     [White, Black, Hispanic, Asian, Other]
SES           [1.6907, -0.3923, 1.1271, -0.4774, 0.1081, 1.0...
base_math     [50.4919, 35.8045, 56.0477, 38.4063, 55.5463, ...
base_level    [6.0, 2.0, 5.0, 3.0, 4.0, 8.0, 1.0, 0.0, 7.0, ...
W1PARENT      [470.25014100000004, 224.455466, 185.301339000...
dtype: object
math          14364
teacher          16
ability          13
parents          11
sex               2
race              5
SES            9060
base_math     14301
base_level       10
W1PARENT       4154
dtype: int64
Generating: lee2021ability
EPSILON: e^-1...
ITERATION: 0...


KeyError: 'lee2021ability'

In [5]:
publication.dataframe.describe()

,math,teacher,ability,parents,SES,base_math,base_level,W1PARENT
count,14575.000000,14575.000000,14575.000000,14575.000000,14575.000000,14575.000000,14575.00000,14575.000000
mean,69.946709,2.233729,2.975077,0.613914,0.148680,42.115971,4.36789,198.277560
std,18.729319,0.525176,0.643115,0.236105,0.789495,11.610209,1.45867,296.955235
min,25.005700,1.000000,1.000000,0.000000,-1.906800,15.864100,0.00000,0.000000
25%,58.915250,1.800000,2.750000,0.500000,-0.427050,34.508650,4.00000,28.870128
50%,69.429700,2.200000,3.000000,0.600000,0.092500,42.208800,4.00000,122.696317
75%,85.156150,2.600000,3.500000,0.800000,0.676300,50.709000,5.00000,276.507522
max,115.100000,4.000000,4.000000,1.000000,2.978300,69.931700,11.00000,7603.292220
